# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import (VectorAssembler,VectorIndexer)
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [2]:
spark = SparkSession.builder.appName('lr_project').getOrCreate()
df = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

24/07/05 14:56:38 WARN Utils: Your hostname, li-0ab60ccc-31c7-11b2-a85c-cfddf3a09b4d.ibm.com resolves to a loopback address: 127.0.0.1; using 192.168.0.247 instead (on interface enp0s31f6)
24/07/05 14:56:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/05 14:56:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/05 14:56:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/05 14:56:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [4]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [6]:
use_cols = df.select([
    'Total_Purchase',
    'Years',
    'Num_Sites',
    'Churn'
 ])

df_clean = use_cols.na.drop()

In [9]:
train_df_clean, test_df_clean = df_clean.randomSplit([0.7,.3])

In [10]:
assembler = VectorAssembler(inputCols=['Total_Purchase','Years','Num_Sites'],outputCol='features')

log_reg = LogisticRegression(featuresCol='features',labelCol='Churn')

pipeline = Pipeline(stages=[assembler,log_reg])

In [11]:
fit_model = pipeline.fit(train_df_clean)

24/07/05 14:56:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [12]:
results = fit_model.transform(test_df_clean)

In [13]:
results.show()

+--------------+-----+---------+-----+-------------------+--------------------+--------------------+----------+
|Total_Purchase|Years|Num_Sites|Churn|           features|       rawPrediction|         probability|prediction|
+--------------+-----+---------+-----+-------------------+--------------------+--------------------+----------+
|        3825.7| 4.28|      8.0|    0|  [3825.7,4.28,8.0]|[4.19266577519373...|[0.98511883230231...|       0.0|
|       4711.89| 4.97|      7.0|    0| [4711.89,4.97,7.0]|[5.02823755279844...|[0.99349228275299...|       0.0|
|       4771.65| 3.77|     12.0|    1|[4771.65,3.77,12.0]|[-0.4193584100020...|[0.39667028717533...|       1.0|
|       4863.73| 5.17|      8.0|    0| [4863.73,5.17,8.0]|[3.71717579972365...|[0.97627409284308...|       0.0|
|        4992.6| 3.85|      8.0|    0|  [4992.6,3.85,8.0]|[4.36192033159738...|[0.98740674037291...|       0.0|
|       5002.58| 4.57|     11.0|    0|[5002.58,4.57,11.0]|[0.38584711763622...|[0.59528257895138...|    

In [14]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [15]:
AUC = my_eval.evaluate(results)
AUC

0.7683409436834094

## Predictions for new dataset

In [16]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [17]:
new_customers.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [29]:
results_new = fit_model.transform(new_customers)

In [31]:
results_new.select(['Company','prediction']).show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

